In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler , LabelEncoder , OneHotEncoder 
import pickle 

In [4]:
data = pd.read_csv("Churn_Modelling.csv")

In [5]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
df = data

In [7]:
df =df.drop(['RowNumber' , 'CustomerId' ,'Surname'], axis = 1)


In [8]:
label_encoder_gender = LabelEncoder()
df['Gender'] = label_encoder_gender.fit_transform(df['Gender'])

In [9]:
oht_encoder_geo = OneHotEncoder()
geo_encoded = oht_encoder_geo.fit_transform(df[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded , columns = oht_encoder_geo.get_feature_names_out(['Geography']))

In [10]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [11]:
df = pd.concat([df.drop('Geography' , axis = 1) , geo_encoded_df] , axis = 1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [12]:
x = df.drop('EstimatedSalary' , axis = 1)
y = df['EstimatedSalary']

In [13]:
x_train  , x_test , y_train , y_test = train_test_split(x , y , test_size=0.25)

In [14]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [15]:
# with open('label_enoder_gender_reg.pkl' , 'wb') as file:
#     pickle.dump(label_encoder_gender,file)
import  pickle
with open('oht_enoder_geo.pkl' ,'wb') as file:
    pickle.dump(oht_encoder_geo,file)
# with open('scaler_reg.pkl' , 'wb') as file:
#     pickle.dump(scaler,file)


In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [17]:
model = Sequential([
    Dense(64 , activation='relu' , input_shape = (x_train.shape[1],)),
    Dense(32 , activation='relu'),
    Dense(1)  # default ACT() is linear
])

c:\Users\bharg\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [18]:
model.compile(optimizer='adam' , loss = 'mean_absolute_error',metrics=['mae'])

In [19]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
from tensorflow.keras.callbacks import EarlyStopping , TensorBoard
import datetime

log_dir = 'Regression/fit/' +datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
Tensorboard_callback = TensorBoard(log_dir = log_dir , histogram_freq = 1)

In [21]:
early_Stopping_callback = EarlyStopping(monitor = 'val_loss' , patience = 10 , restore_best_weights=True)

In [20]:
history = model.fit(
    x_train , y_train,
    validation_data = (x_test , y_test),
    epochs = 100 ,
    callbacks = [early_Stopping_callback , Tensorboard_callback]
)

Epoch 1/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - loss: 99620.6641 - mae: 99620.6641 - val_loss: 101194.8594 - val_mae: 101194.8594
Epoch 2/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 98998.2891 - mae: 98998.2891 - val_loss: 99923.6016 - val_mae: 99923.6016
Epoch 3/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 96755.6406 - mae: 96755.6406 - val_loss: 96597.6250 - val_mae: 96597.6250
Epoch 4/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 92258.8438 - mae: 92258.8438 - val_loss: 90906.3516 - val_mae: 90906.3516
Epoch 5/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 85573.1016 - mae: 85573.1016 - val_loss: 83214.0000 - val_mae: 83214.0000
Epoch 6/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 77326.3438 - mae: 77326.3438 - val_loss: 74644.5703 - val_mae: 74644.5703
Epoch 7/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 68821.2578 - mae: 68821.2578 - val_loss: 66606.6328 - val_mae: 66606.6328
Epoch 8/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step

In [21]:
%load_ext tensorboard

In [22]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 1700), started 11:32:09 ago. (Use '!kill 1700' to kill it.)

In [23]:
test_loss , test_mae = model.evaluate(x_test , y_test)
print(f"reset MAE : {test_mae}")

79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 50244.8555 - mae: 50244.8555
reset MAE : 50244.85546875


In [24]:
model.save('regression.h5')